### Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import os
import pickle 

In [2]:
import interpret
import shap
from interpret.blackbox import ShapKernel, LimeTabular

c:\Users\Sebastian's work\OneDrive - OptiMedis AG\Dokumente\Coding\High-Cost-patient-analysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [41]:
path_list = ('results/', 'complete/', 'unbalanced/', 'model_explanation/', 'interpretML')
cur_path = ''
for rep in path_list:
    cur_path += rep
    if not os.path.exists(cur_path):
        os.makedirs(cur_path)
result_path = ''.join(path_list)

# CREATE EXPLANATIONS FOR RANDOM FOREST 
**File**: 5_explanation_interpretml.ipynb

**Author**: Sebastian Benno Veuskens 

**Date**: 2024-07-28

**Data**: train and test (same as evaluated model)  


## Modify

**Working directory**

In [3]:
os.chdir("C:/Users/Sebastian's work/OneDrive - OptiMedis AG/Dokumente/Coding/High-Cost-patient-analysis")

**Parameters & Settings**

In [14]:
target = 'HC_Patient_Next_Year'
excluded = 'Total_Costs_Next_Year'
prediction_outcome = 1  # Indicate which prediction outcome the sample should have 
true_outcome = 1 # Indicate which true HCP status the patient in the sample should have

### Load data

In [5]:
train = pd.read_csv('data/complete/unbalanced/train_validate.csv', sep=',', header=0)
test = pd.read_csv('data/complete/unbalanced/test.csv', sep=',', header=0)

### Transform categorical values

In [6]:
# TODO: Check if I need this or whether I need to make the same transformations as before
# train['Sex'] = pd.factorize(train['Sex'])[0]
# test['Sex'] = pd.factorize(test['Sex'])[0]

In [7]:
from sklearn.preprocessing import OrdinalEncoder
all_columns = train.columns

# If a column has more than two values, it is not categorical
categorical_columns = all_columns[train.nunique(axis=0) == 2]
numerical_columns = [all_columns.difference(categorical_columns)]

categorical_columns_indices = [test.columns.tolist().index(cn) for cn in categorical_columns]
oe = OrdinalEncoder().fit(train[categorical_columns])
categorical_name_mapping = {i: list(v) if i < 11 else ['diagnosis absent', 'diagnosis present']
                            for (i, v) in zip(categorical_columns_indices, oe.categories_)}


train.loc[:, categorical_columns] = oe.transform(train[categorical_columns])
test.loc[:, categorical_columns] = oe.transform(test[categorical_columns])

### Split predictors & outcome labels 

In [8]:
predictors = [var for var in train.columns if var not in (target, excluded)]
X_train, y_train = train[predictors], train[target]
X_test, y_test = test[predictors], test[target]

In [9]:
train.head()

,HC_Patient_Next_Year,Total_Costs_Next_Year,HC_Patient,Sex,Age,Need_of_Care_Duration,DMP_Duration,Total_Costs,Inpatient_Num_Diagnoses,Outpatient_Num_Diagnoses,...,Prescription_S03,Prescription_V01,Prescription_V03,Prescription_V04,Prescription_V06,Prescription_V07,Prescription_V08,Prescription_V09,Prescription_V10,Prescription_V70
0,0,30.508696,0,0.0,31,0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,30.571859,0,0.0,28,0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,30.898212,0,0.0,28,0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,30.898212,0,0.0,47,0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,30.909809,0,0.0,34,0,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Load Model

In [10]:
with open('results/complete/unbalanced/model_explanation/random_forest_python.sav', 'rb') as handle:
    model = pickle.load(handle)
    
with open('results/complete/unbalanced/model_explanation/random_forest_python_threshold.sav', 'rb') as handle:
    best_thresh = pickle.load(handle)

In [11]:
best_thresh

0.07

### Choose sample
Sample to investigate better, choose sample with user-specified model outcome

In [12]:
prediction_probs = model.predict_proba(X_test)[:,1]
prediction_probs_pos = prediction_probs[y_test == 1]
prediction_probs_neg = prediction_probs[y_test == 0]

predictions = np.array(prediction_probs >= best_thresh, dtype=int)
predictions_pos = np.array(prediction_probs_pos >= best_thresh, dtype=int)
predictions_neg = np.array(prediction_probs_neg >= best_thresh, dtype=int)

In [15]:
samples_selected = X_test.loc[(y_test == true_outcome) & (predictions == prediction_outcome)]
samples_true_pos = X_test.loc[(y_test == 1) & (predictions == 1)]
samples_false_pos = X_test.loc[(y_test == 0) & (predictions == 1)]
samples_false_neg = X_test.loc[(y_test == 1) & (predictions == 0)]
samples_true_neg = X_test.loc[(y_test == 0) & (predictions == 0)]

## XAI METHODS

### MODIFY

In [23]:
pd.concat([samples_true_pos.iloc[:2], samples_true_neg.iloc[:2]])

,HC_Patient,Sex,Age,Need_of_Care_Duration,DMP_Duration,Total_Costs,Inpatient_Num_Diagnoses,Outpatient_Num_Diagnoses,Prescription_Num_Prescriptions,Inpatient_A,...,Prescription_S03,Prescription_V01,Prescription_V03,Prescription_V04,Prescription_V06,Prescription_V07,Prescription_V08,Prescription_V09,Prescription_V10,Prescription_V70
386,0,1.0,56,0,0,4876.174026,2,41,15,0,...,0,0,0,0,0,0,0,0,0,0
417,0,1.0,70,1,0,5382.526709,3,77,38,0,...,0,0,0,0,0,0,0,0,0,0
0,0,1.0,73,0,0,152.313555,0,35,6,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0.0,12,0,0,152.377651,0,5,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# Indicate the indices of the samples I would like to explain
local_samples_idx = list(samples_true_pos.index[:2]) + list(samples_false_pos.index[:2]) + list(samples_false_neg.index[:2]) + list(samples_true_neg.index[:2])
local_samples_X = X_test.iloc[local_samples_idx]
local_samples_y = y_test.iloc[local_samples_idx]
local_samples_predictions = predictions[local_samples_idx]

### SHAP 

In [33]:
# Due to size of data set, summarization techniques are required
# TODO: Figure out a good summarize number here 
X_train_summary = shap.sample(X_train, 200)
exp_shap = ShapKernel(model, X_train_summary)

Using 200 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


In [34]:
# shap = ShapKernel(model, X_train)
exp_local_shap = exp_shap.explain_local(local_samples_X, local_samples_y)

100%|██████████| 8/8 [1:39:14<00:00, 744.34s/it]   


In [35]:
interpret.show(exp_local_shap)

<!-- http://127.0.0.1:7001/2232310283056/ -->

### LIME 

In [36]:
exp_lime = LimeTabular(model, X_train)
exp_local_lime = exp_lime.explain_local(local_samples_X, local_samples_y)

In [39]:
# TODO: What does the 0 do here? 
interpret.show(exp_local_lime, 1)

<!-- http://127.0.0.1:7001/2236866970976/ -->

### PDP 

In [42]:
pdp_iml = interpret.blackbox.PartialDependence(model, X_train)
pdp_global_iml = pdp_iml.explain_global()

In [ ]:
# TODO: Check why there is a one in interpret.show(pdp...)
exp_key = 0
interpret.show(pdp_iml.explain_global(), exp_key)

NameError: name 'pdp_iml' is not defined

### Morris Sensitivity Analysis 

In [65]:
msa_iml = interpret.blackbox.MorrisSensitivity(model, X_train)
msa_global_iml = msa_iml.explain_global()

c:\Users\Sebastian's work\OneDrive - OptiMedis AG\Dokumente\Coding\High-Cost-patient-analysis\.venv\Lib\site-packages\SALib\util\__init__.py:302: FutureWarning:

unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.



MemoryError: Unable to allocate 3.18 MiB for an array with shape (646, 645) and data type float64

In [ ]:
interpret.show(msa_global_iml, 0)

### Dashboard 

In [ ]:
interpret.show([exp_local_shap, exp_local_lime, pdp_global_iml])